In [31]:
# Activate Spark in our Colab notebook.
import os


In [32]:
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'

In [33]:
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

In [34]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (102 kB/s)
Reading package lis

In [35]:
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

In [36]:
# Start a SparkSession
import findspark
findspark.init()

In [64]:
# Start Spark session
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from pyspark.sql import DataFrameNaFunctions
spark = SparkSession.builder.appName("group_3_midterm_pySpark").getOrCreate()

In [65]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
#url = "https://2u-data-curriculum-team.s3.amazonaws.com/nflx-data-science-adv/week-4/food.csv"
#spark.sparkContext.addFile(url)
db_df = spark.read.csv("diabetic_data.csv", sep=",", header=True)

# Show DataFrame
db_df.show()

+------------+-----------+---------------+------+--------+------+-----------------+------------------------+-------------------+----------------+----------+--------------------+------------------+--------------+---------------+-----------------+----------------+----------------+------+------+------+----------------+-------------+---------+---------+-----------+-----------+--------------+-----------+-------------+---------+---------+-----------+------------+-------------+--------+--------+------------+----------+-------+-----------+-------+-------------------+-------------------+------------------------+-----------------------+----------------------+------+-----------+----------+
|encounter_id|patient_nbr|           race|gender|     age|weight|admission_type_id|discharge_disposition_id|admission_source_id|time_in_hospital|payer_code|   medical_specialty|num_lab_procedures|num_procedures|num_medications|number_outpatient|number_emergency|number_inpatient|diag_1|diag_2|diag_3|number_diagn

In [66]:
#Look at the information on the datframe (the data types and the number of rows)
db_df.printSchema()

root
 |-- encounter_id: string (nullable = true)
 |-- patient_nbr: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- admission_type_id: string (nullable = true)
 |-- discharge_disposition_id: string (nullable = true)
 |-- admission_source_id: string (nullable = true)
 |-- time_in_hospital: string (nullable = true)
 |-- payer_code: string (nullable = true)
 |-- medical_specialty: string (nullable = true)
 |-- num_lab_procedures: string (nullable = true)
 |-- num_procedures: string (nullable = true)
 |-- num_medications: string (nullable = true)
 |-- number_outpatient: string (nullable = true)
 |-- number_emergency: string (nullable = true)
 |-- number_inpatient: string (nullable = true)
 |-- diag_1: string (nullable = true)
 |-- diag_2: string (nullable = true)
 |-- diag_3: string (nullable = true)
 |-- number_diagnoses: string (nullable = true)
 |-- max_glu_ser

# **ETL Process**

In [67]:
#drop weight, payor_code, medical speacialty
df_dropped_col_3 = db_df.drop('weight','medical_specialty','payer_code')
df_dropped_col_3

DataFrame[encounter_id: string, patient_nbr: string, race: string, gender: string, age: string, admission_type_id: string, discharge_disposition_id: string, admission_source_id: string, time_in_hospital: string, num_lab_procedures: string, num_procedures: string, num_medications: string, number_outpatient: string, number_emergency: string, number_inpatient: string, diag_1: string, diag_2: string, diag_3: string, number_diagnoses: string, max_glu_serum: string, A1Cresult: string, metformin: string, repaglinide: string, nateglinide: string, chlorpropamide: string, glimepiride: string, acetohexamide: string, glipizide: string, glyburide: string, tolbutamide: string, pioglitazone: string, rosiglitazone: string, acarbose: string, miglitol: string, troglitazone: string, tolazamide: string, examide: string, citoglipton: string, insulin: string, glyburide-metformin: string, glipizide-metformin: string, glimepiride-pioglitazone: string, metformin-rosiglitazone: string, metformin-pioglitazone: s

In [112]:
#replace question mark signs with Na
#df_w_nulls=df_dropped_col_3.replace("?",None).show()
df_w_nulls=df_dropped_col_3.na.fill(value = "?", subset = None)

In [113]:
df_w_nulls.show()

+------------+-----------+---------------+------+--------+-----------------+------------------------+-------------------+----------------+------------------+--------------+---------------+-----------------+----------------+----------------+------+------+------+----------------+-------------+---------+---------+-----------+-----------+--------------+-----------+-------------+---------+---------+-----------+------------+-------------+--------+--------+------------+----------+-------+-----------+-------+-------------------+-------------------+------------------------+-----------------------+----------------------+------+-----------+----------+
|encounter_id|patient_nbr|           race|gender|     age|admission_type_id|discharge_disposition_id|admission_source_id|time_in_hospital|num_lab_procedures|num_procedures|num_medications|number_outpatient|number_emergency|number_inpatient|diag_1|diag_2|diag_3|number_diagnoses|max_glu_serum|A1Cresult|metformin|repaglinide|nateglinide|chlorpropamide|

In [85]:
from pyspark.sql import functions

In [96]:
#drop null values and reset the index
#df_w_nulls.na.drop("any").show(false)
#new=df_w_nulls.na.drop(how="all", inplace=False)
dropped_df=df_w_nulls.na.drop()
#dropped_df=dropped_df.drop(columns=['index'])
dropped_df.show()

+------------+-----------+---------------+------+--------+-----------------+------------------------+-------------------+----------------+------------------+--------------+---------------+-----------------+----------------+----------------+------+------+------+----------------+-------------+---------+---------+-----------+-----------+--------------+-----------+-------------+---------+---------+-----------+------------+-------------+--------+--------+------------+----------+-------+-----------+-------+-------------------+-------------------+------------------------+-----------------------+----------------------+------+-----------+----------+
|encounter_id|patient_nbr|           race|gender|     age|admission_type_id|discharge_disposition_id|admission_source_id|time_in_hospital|num_lab_procedures|num_procedures|num_medications|number_outpatient|number_emergency|number_inpatient|diag_1|diag_2|diag_3|number_diagnoses|max_glu_serum|A1Cresult|metformin|repaglinide|nateglinide|chlorpropamide|

In [97]:
dropped_df.info()

AttributeError: ignored

### Break down the Master dataset into normalized tables.
# Patient
*   patient_nbr (PK)
*   age
*   gender
*   race
# Admission_type
*   admission_type_id (PK)
*   admission_name
# Admission source
*   admission_source_id (PK)
*   asmission_source_name
# Discharge Table
-discharge_disposition_id (PK)
-discharge_name
# Medication Table
-medication__id (PK)
-medication_type
# Patient_medication Table
-medication__id (PK)
-encounter_id (PK)
-change_in_dosage
# Diagnosis Table
-diagnosis_id
-diagnosis_type
# Encounter Table
-encounter_id (PK)
-patient_nbr (FK)
-num_lab_procedures
-num_procedures
-num_medications
-number_emergency
-number_outpatient
-number_inpatient
-time_in_hospital
-readmitted
-admission_type_id (FK)
-admission_source_id (FK)
-discharge_disposition_id (FK)
-diag_1(FK)
-diag_2(FK)
-diag_3(FK)
-number_diagnoses
-change
-A1Cresult
-max_glu_serum
-diabetesMed







In [ ]:
#create a patient table with unique patient number to remove duplicates
def patient_tab(df):
  uniq_patient_number=df.drop_duplicates(subset = "patient_nbr")
  patient_table=uniq_patient_number[['patient_nbr','age','race','gender']].reset_index().set_index('patient_nbr').drop(columns=['index'])
  return patient_table

In [ ]:
#call on the patient tab fundtion to see the patient table
patient_table=patient_tab(dropped_df)
patient_table.head()

In [ ]:
patient_table.to_csv('Patient_Table.csv')

In [ ]:
#read in the admission type csv file 
admission_type=pd.read_csv('admission_type.csv')
admission_type.head()

In [ ]:
#read in the admission source csv file
admission_source=pd.read_csv('admission_source.csv')
admission_source.info()

In [ ]:
#read in the discharge csv file
discharge_disposition=pd.read_csv('discharge_disposition.csv')
discharge_disposition.head()

In [ ]:
#Create Medication table
def medic_tab(df):
  medication_id=np.arange(1,24)
  medication_type=df.columns.values[21:44]
  Medication=pd.DataFrame({"medication_id":medication_id, "medication_type":medication_type})
  return(Medication)

In [ ]:
#Preview the medication tab
Medication=medic_tab(dropped_df)
Medication.head()

In [ ]:
Medication.to_csv('Medication Table.csv')

In [ ]:
df=dropped_df

In [ ]:
#Create Patient medication table
#def patient_medic(df):
#create two empty dataframes to update the new information with
patient_med = pd.DataFrame()
patient_med_up = pd.DataFrame()
#loop through medication type to get the corresponing columns
for i in Medication['medication_type']:
  if i=='metformin':
#retrieve the encounter id for all meedications that were changed
patient_med['encounter_id']=df['encounter_id'].where((df[i]=='Down') | (df[i]=='Up') | (df[i]=='Steady')).dropna()
#assign medical Id number for all medication types
patient_med['medication_id']=i
patient_med['medication_id'] = Medication['medication_id'].where(Medication['medication_type']==i).dropna()[0]
#retrieve the change in dosage
patient_med['change_in_dosage']=df[i].where((df[i]=='Up') | (df[i]=='Down') | (df[i]=='Steady')).dropna()
elif i!='metformin':
patient_med_up['encounter_id']=df['encounter_id'].where((df[i]=='Down') | (df[i]=='Up') | (df[i]=='Steady')).dropna()
patient_med_up['medication_id']=i
patient_med_up['medication_id'] = Medication['medication_id'].where(Medication['medication_type']==i).dropna().values[0]
patient_med_up['change_in_dosage']=df[i].where((df[i]=='Up') | (df[i]=='Down') | (df[i]=='Steady')).dropna()
patient_medication=patient_med.append(patient_med_up).dropna()


In [ ]:
patient_medication

In [ ]:
#Preview patient medication table
patient_medication=patient_medic(dropped_df)
patient_medication.head()

In [ ]:
#create the dosage table
diag_df=pd.read_csv('ICD-9_Code_1998.csv', sep=',')
#split columns into to two
diag_df_new=diag_df['ICD TITLE & CODE CROSS REFERENCE FILE'].str.split('\s+-\d|\s+_1', expand=True)
#drop empty columns and rename the columns
diag_df_n=diag_df_new.drop(columns=[2]).dropna()
diag_df_n.head()

In [ ]:
#clean catgeories
strs=[]
for i in diag_df_n[1]:
  try:
    if '-' in i:
      strs.append(i)
    pd.to_numeric(i, downcast='float')
  except:
    if i[0]!='E':
      strs.append(i)
diag_df_cln = diag_df_n.copy(deep=True)
for i in strs:
  diag_df_cln = diag_df_cln.where(diag_df_cln[1]!=i)
diag_df_cln=diag_df_cln.dropna()

In [ ]:
#delete rows with category data
diag_df_cln = diag_df_n.copy(deep=True)
for i in strs:
  diag_df_cln = diag_df_cln.where(diag_df_cln[1]!=i).dropna()
# diag_df_cln=diag_df_cln
diagnosis_table=diag_df_cln.rename(columns={0:"diagnosis_type",1:"diagnosis_code"}).reset_index().drop(columns=['index'])

In [ ]:
#preview diagnosis table
diagnosis_table.to_csv('Diagnosis Table.csv')
diagnosis_table['diagnosis_type'].where

In [ ]:
#create an encounter Table
def encount(df):
  new_patient_table=np.delete(patient_table.columns.values, [0])
  values=[]
  for i in df.columns.values:
    if (i in Medication.medication_type.values) or (i in new_patient_table):
      values.append(i)
  Encounter=dropped_df.drop(columns=values)
  return (Encounter)

In [ ]:
#preview encounter table
Encounter = encount(dropped_df)
Encounter.to_csv('Encounter table.csv')
Encounter.head()

# **Quality assurance process**

#Assertion test 1

In [ ]:
def len_patient_table(df):
  len(df.columns)==50

In [ ]:
def assert_patient_table_pass():
  assert len_patient_table(dropped_df)==50

In [ ]:
def assert_patient_table_1():
  assert len_patient_table(dropped_df)==40

# Assertion test 2

In [ ]:
def check_meds(df):
  assert np.array_equal(Medication.medication_type.values, df.columns.values[24:47])

In [ ]:
db_df.columns.values[24:47]

In [ ]:
check_meds(db_df)

In [ ]:
def assert_patient_medication():
  assert patient_medication['medication_id'].nunique()==23

In [ ]:
assert_patient_medication()